In [45]:
import pandas as pd
import numpy as np
import os

In [51]:
def calculate_returns(df):
    """
    Calculate percentage returns for each column
    Returns original dataframe with 4 new columns for returns
    """
    # Calculate percentage returns
    returns = df.pct_change()
    
    # Add suffix '_ret' to column names for returns
    returns = returns.add_suffix('_ret')
    
    # Concatenate original data with returns
    result = pd.concat([df, returns], axis=1)
    
    return result

# Using the previous dataframe
result = calculate_returns(result)

C:\Users\westm\AppData\Local\Temp\ipykernel_23644\3798901245.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df.pct_change()


In [61]:
def process_economic_data(files, rate_file):
    """
    Process economic data files and interest rate data, converting all data to daily frequency.
    Fills gaps between existing data points while preserving NA values before the first data point.
    """
    # Dictionary for more readable column names
    column_names = {
        'GDP': 'GDP',
        'INDPRO': 'Industrial_Production',
        'RSXFS': 'Retail_Sales',
        'W790RC1Q027SBEA': 'Wage_and_Salary',
        'CPIAUCSL': 'Consumer_Price_Index',
        'M2SL': 'Money_Supply_M2',
        'PCE': 'Personal_Consumption',
        'PPIACO': 'Producer_Price_Index',
        'CES0500000003': 'Average_Hourly_Earnings',
        'JTSJOL': 'Job_Openings',
        'UNRATE': 'Unemployment_Rate',
        'COMREPUSQ159N': 'Commercial_Real_Estate_Prices',
        'MSPUS': 'Median_House_Price',
        'T10Y2Y': 'Treasury_Spread_10Y_2Y',
        'NASDAQCOM': 'Nasdaq_Composite',
        'FEDFUNDS': 'Federal_Funds_Rate'
    }
    
    # Read all files
    dfs = {}
    for file in files + [rate_file]:
        try:
            name = file.split('\\')[-1].split('.')[0]
            df = pd.read_csv(file)
            # Convert to numeric, coerce errors to NaN
            value_col = df.columns[1]  # Assume second column is the value column
            df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
            df['DATE'] = pd.to_datetime(df['DATE'])
            df = df.set_index('DATE')
            
            # Rename column to readable name
            if name in column_names:
                df.columns = [column_names[name]]
            
            dfs[name] = df
            print(f"Successfully loaded: {name} as {df.columns[0]}")
        except Exception as e:
            print(f"Error loading {name}: {str(e)}")
            continue

    # Convert all data to daily frequency
    processed_dfs = {}
    for name, df in dfs.items():
        try:
            # Get the full date range from the earliest to latest date across all datasets
            start_date = min([df.index.min() for df in dfs.values()])
            end_date = max([df.index.max() for df in dfs.values()])
            
            # Create a daily date range
            daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
            
            # Store the first valid index for this dataset
            first_valid_index = df.first_valid_index()
            
            # Reindex the data to daily frequency
            daily_df = df.reindex(daily_index)
            
            # Create a mask for all dates after the first valid data point
            mask = daily_df.index >= first_valid_index
            
            # For dates after first valid data point:
            # 1. Forward fill
            # 2. Back fill any remaining gaps
            valid_period = daily_df.loc[mask]
            valid_period = valid_period.fillna(method='ffill')
            valid_period = valid_period.fillna(method='bfill')
            
            # Combine the filled valid period with the NA period before first data point
            daily_df.loc[mask] = valid_period
            
            processed_dfs[name] = daily_df
            print(f"{name} processed to daily frequency")
        except Exception as e:
            print(f"Error processing {name}: {str(e)}")
            continue
    
    # Merge all dataframes
    result = pd.concat(processed_dfs.values(), axis=1)
    
    # Add some data quality checks
    missing_data = result.isnull().sum()
    if missing_data.any():
        print("\nMissing data report:")
        print(missing_data[missing_data > 0])
    
    # Calculate the percentage of available data points
    # Only consider the period after first valid data point for each series
    data_availability = {}
    for column in result.columns:
        first_valid = result[column].first_valid_index()
        if first_valid is not None:
            valid_period = result.loc[first_valid:, column]
            availability = (valid_period.count() / len(valid_period)) * 100
            data_availability[column] = availability
    
    print("\nData availability percentage (after first valid data point):")
    for column, availability in data_availability.items():
        print(f"{column}: {availability:.2f}%")
    
    return result

# Data Processing

In [62]:
files = [
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Economic Growth\GDP.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Economic Growth\INDPRO.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Economic Growth\RSXFS.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Economic Growth\W790RC1Q027SBEA.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Inflation\CPIAUCSL.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Inflation\M2SL.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Inflation\PCE.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Inflation\PPIACO.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Job Market\CES0500000003.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Job Market\JTSJOL.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Job Market\UNRATE.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Financial Conditions\COMREPUSQ159N.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Financial Conditions\MSPUS.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Financial Conditions\T10Y2Y.csv",
    r"C:\Users\westm\Trading\Interest rate forecast\Data\Financial Conditions\NASDAQCOM.csv"
]

rate_file = r"C:\Users\westm\Trading\Interest rate forecast\Data\Interest Rate\FEDFUNDS.csv"

In [63]:
# Process all data
full_data = process_economic_data(files, rate_file)

# Print information about the dataset
print("\nDataset info:")
print(f"Date range: {full_data.index.min()} to {full_data.index.max()}")
print(f"Number of rows: {len(full_data)}")
print("\nColumns in dataset:")
print(full_data.columns.tolist())

print("\nMissing values per column:")
print(full_data.isna().sum())

print("\nData availability by series:")
for col in full_data.columns:
    first_valid = full_data[col].first_valid_index()
    last_valid = full_data[col].last_valid_index()
    print(f"\n{col}:")
    print(f"First available data: {first_valid}")
    print(f"Last available data: {last_valid}")

print("\nFirst few rows of data:")
print(full_data.head())

Successfully loaded: GDP as GDP
Successfully loaded: INDPRO as Industrial_Production
Successfully loaded: RSXFS as Retail_Sales
Successfully loaded: W790RC1Q027SBEA as Wage_and_Salary
Successfully loaded: CPIAUCSL as Consumer_Price_Index
Successfully loaded: M2SL as Money_Supply_M2
Successfully loaded: PCE as Personal_Consumption
Successfully loaded: PPIACO as Producer_Price_Index
Successfully loaded: CES0500000003 as Average_Hourly_Earnings
Successfully loaded: JTSJOL as Job_Openings
Successfully loaded: UNRATE as Unemployment_Rate
Successfully loaded: COMREPUSQ159N as Commercial_Real_Estate_Prices
Successfully loaded: MSPUS as Median_House_Price
Successfully loaded: T10Y2Y as Treasury_Spread_10Y_2Y
Successfully loaded: NASDAQCOM as Nasdaq_Composite
Successfully loaded: FEDFUNDS as Federal_Funds_Rate
GDP processed to daily frequency
INDPRO processed to daily frequency
RSXFS processed to daily frequency
W790RC1Q027SBEA processed to daily frequency
CPIAUCSL processed to daily frequency


C:\Users\westm\AppData\Local\Temp\ipykernel_23644\1689889599.py:72: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  valid_period = valid_period.fillna(method='ffill')
C:\Users\westm\AppData\Local\Temp\ipykernel_23644\1689889599.py:73: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  valid_period = valid_period.fillna(method='bfill')


GDP                              12418
Industrial_Production             2191
Retail_Sales                     28854
Wage_and_Salary                  17166
Consumer_Price_Index             12418
Money_Supply_M2                  16801
Personal_Consumption             16801
Producer_Price_Index                 0
Average_Hourly_Earnings          34027
Job_Openings                     32111
Unemployment_Rate                12783
Commercial_Real_Estate_Prices    33603
Median_House_Price               18262
Treasury_Spread_10Y_2Y           23162
Nasdaq_Composite                 21219
Federal_Funds_Rate               15156
dtype: int64

Data availability by series:

GDP:
First available data: 1947-01-01 00:00:00
Last available data: 2024-10-23 00:00:00

Industrial_Production:
First available data: 1919-01-01 00:00:00
Last available data: 2024-10-23 00:00:00

Retail_Sales:
First available data: 1992-01-01 00:00:00
Last available data: 2024-10-23 00:00:00

Wage_and_Salary:
First available data:

In [64]:
# Save to CSV
save_dir = r"C:\Users\westm\Trading\Interest rate forecast\Data\Clean Data"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "DATACLEAN.csv")
full_data.to_csv(save_path)

print(f"\nData saved to: {save_path}")
print(f"File size: {os.path.getsize(save_path) / (1024*1024):.2f} MB")
print(f"Number of rows: {len(full_data)}")
print(f"Number of columns: {len(full_data.columns)}")


Data saved to: C:\Users\westm\Trading\Interest rate forecast\Data\Clean Data\DATACLEAN.csv
File size: 3.11 MB
Number of rows: 40839
Number of columns: 16
